In [3]:
!pip install -q flwr[simulation] torch torchvision

In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import time
import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 1.13.1 and Flower 1.4.0


In [2]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):  # Use the passed 'epochs' variable here
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()  # Make sure to call .item() to get the scalar value
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch}: train loss {epoch_loss:.6f}, accuracy {epoch_acc:.6f}")



def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [10]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.runtime = 0.0

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        global runtime
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        # Use the 'epochs' value from the config dictionary, defaulting to 1 if it's not specified
        epochs = config.get("epochs", 1)
        training_start = time.time()
        train(self.net, self.trainloader, epochs)
        
        self.runtime = time.time() - training_start
        print(f"Training Finished in {self.runtime}")

        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client {self.cid} loss {loss}")
        print(f"Client {self.cid} accuracy {accuracy}")
        
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [8]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.runtime = 0.0  # Variable to store the training runtime

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        
        # Create custom configs
        n_clients = len(clients)
        half_clients = n_clients // 2
        standard_config = {"lr": 0.001, "epochs": 3}
        higher_lr_config = {"lr": 0.003, "epochs": 5}
        fit_configurations = []

        for idx, client in enumerate(clients):
            if idx < half_clients:
                fit_configurations.append((client, FitIns(parameters, standard_config)))
            else:
                fit_configurations.append(
                    (client, FitIns(parameters, higher_lr_config))
                )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients


In [9]:
# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-05-02 17:22:55,597 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


INFO flwr 2024-05-02 17:23:01,008 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 36687611904.0, 'memory': 75604427776.0, 'GPU': 1.0, 'CPU': 32.0}
INFO flwr 2024-05-02 17:23:01,011 | server.py:86 | Initializing global parameters
INFO flwr 2024-05-02 17:23:01,022 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-05-02 17:23:01,024 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-05-02 17:23:01,026 | server.py:101 | FL starting
DEBUG flwr 2024-05-02 17:23:01,028 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=26224) [Client 2] fit, config: {'lr': 0.001, 'epochs': 3}
 pid=26224) Epoch 0: train loss 0.065225, accuracy 0.214444
 pid=26224) Epoch 1: train loss 0.057076, accuracy 0.316222
 pid=26224) Epoch 2: train loss 0.052112, accuracy 0.391556
 pid=26224) Training Finished in  19.601341724395752
 pid=26224) [Client 1] fit, config: {'lr': 0.003, 'epochs': 5}
 pid=26224) Epoch 0: train loss 0.064412, accuracy 0.230222
 pid=26224) Epoch 1: train loss 0.055319, accuracy 0.351556
 pid=26224) Epoch 2: train loss 0.051261, accuracy 0.401556
 pid=26224) Epoch 3: train loss 0.048549, accuracy 0.434444
 pid=26224) Epoch 4: train loss 0.046451, accuracy 0.461333
 pid=26224) Training Finished in  13.312281608581543
 pid=26224) [Client 0] fit, config: {'lr': 0.003, 'epochs': 5}
 pid=26224) Epoch 0: train loss 0.065064, accuracy 0.236000
 pid=26224) Epoch 1: train loss 0.056888, accuracy 0.329778
 pid=26224) Epoch 2: train loss 0.051560, accuracy 0.395556
 pid=26224) Epoch 3: train loss 0.049039, acc

DEBUG flwr 2024-05-02 17:24:18,755 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-05-02 17:24:18,777 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=26224) Epoch 2: train loss 0.051427, accuracy 0.390889
 pid=26224) Training Finished in  8.002503633499146
 pid=26224) [Client 0] evaluate, config: {}
 pid=26224) Client 0 loss 0.055161301136016846
 pid=26224) Client 0 accuracy 0.37
 pid=26224) [Client 1] evaluate, config: {}
 pid=26224) Client 1 loss 0.05531983661651611
 pid=26224) Client 1 accuracy 0.402
 pid=26224) [Client 3] evaluate, config: {}
 pid=26224) Client 3 loss 0.05547668504714966
 pid=26224) Client 3 accuracy 0.392
 pid=26224) [Client 2] evaluate, config: {}


...

DEBUG flwr 2024-05-02 17:34:52,620 | server.py:232 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-05-02 17:34:52,642 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 5)


 pid=26224) Epoch 4: train loss 0.009123, accuracy 0.909111
 pid=26224) Training Finished in  13.424917697906494
 pid=26224) [Client 2] evaluate, config: {}
 pid=26224) Client 2 loss 0.051071773290634156
 pid=26224) Client 2 accuracy 0.558
 pid=26224) [Client 0] evaluate, config: {}
 pid=26224) Client 0 loss 0.049999896883964536
 pid=26224) Client 0 accuracy 0.534
 pid=26224) [Client 1] evaluate, config: {}
 pid=26224) Client 1 loss 0.051228297114372254
 pid=26224) Client 1 accuracy 0.572
 pid=26224) [Client 4] evaluate, config: {}
 pid=26224) Client 4 loss 0.04735017490386963
 pid=26224) Client 4 accuracy 0.546
 pid=26224) [Client 3] evaluate, config: {}


DEBUG flwr 2024-05-02 17:35:00,685 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-05-02 17:35:00,687 | server.py:147 | FL finished in 719.6606386999999
INFO flwr 2024-05-02 17:35:00,688 | app.py:218 | app_fit: losses_distributed [(1, 0.055289751577377325), (2, 0.045957988309860226), (3, 0.04286436624526977), (4, 0.04095723600387573), (5, 0.04051456909179687), (6, 0.04156583504676818), (7, 0.04266153445243835), (8, 0.043990607500076294), (9, 0.04673468129634857), (10, 0.04998565466403961)]
INFO flwr 2024-05-02 17:35:00,689 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-05-02 17:35:00,691 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-05-02 17:35:00,692 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-05-02 17:35:00,693 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.055289751577377325
	round 2: 0.045957988309860226
	round 3: 0.04286436624526977
	round 4: 0.04095723600387573
	round 5: 0.04051456909179687
	round 6: 0.04156583504676818
	round 7: 0.04266153445243835
	round 8: 0.043990607500076294
	round 9: 0.04673468129634857
	round 10: 0.04998565466403961

 pid=26224) Client 3 loss 0.05027813112735748
 pid=26224) Client 3 accuracy 0.576


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(ClientAppActor pid=10645) [Client 1] evaluate, config: {}


(ClientAppActor pid=10645) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
